In [1]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import logging

# Module importieren
from utils.logging_setup import setup_logging
from data.data_loader import load_price_data
from models.energy_storage import EnergyStorage
from strategies.threshold_lookahead import threshold_lookahead
import config

def test_window_sizes():
    """
    Testet verschiedene Fenstergrößen für die Threshold-Lookahead-Strategie und
    analysiert deren Einfluss auf den Arbitragegewinn.
    """
    # Logger setup
    logger = setup_logging()
    logger.info("Starte Analyse verschiedener Window Sizes für die Threshold-Lookahead-Strategie")
    
    # Basisverzeichnis für die Ausgabe
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_output_dir = os.path.join(config.OUTPUT_DIR, f"window_analysis_{timestamp}")
    os.makedirs(base_output_dir, exist_ok=True)
    
    # Preisdaten laden
    prices_df = load_price_data(config.DEFAULT_PRICE_FILE)
    if prices_df is None:
        logger.error("Fehler beim Laden der Daten. Bitte Dateipfad überprüfen.")
        return
    
    # Definition der zu testenden Window Sizes
    # 0 = keine Lookahead, 4 = 1 Stunde, 96 = 24 Stunden (bei 15-min Intervallen)
    window_sizes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                    16, 17, 18, 19, 20]
    
    # Ergebnisse speichern
    results = []
    
    # Startzeit für Gesamtlaufzeit
    total_start_time = time.time()
    
    # Jede Window Size testen
    for window_size in window_sizes:
        logger.info(f"\n{'='*80}")
        logger.info(f"TESTE WINDOW SIZE: {window_size} (entspricht {window_size*15} Minuten)")
        logger.info(f"{'='*80}")
        
        # Spezifischen Ausgabeordner für diese Window Size erstellen
        window_dir = os.path.join(base_output_dir, f"window_{window_size}")
        os.makedirs(window_dir, exist_ok=True)
        
        # Startzeit für diese Simulation
        start_time = time.time()
        
        try:
            # Speicher initialisieren
            storage = EnergyStorage(
                capacity_mwh=config.CAPACITY_MWH,
                charge_rate=config.CHARGE_DISCHARGE_RATE,
                efficiency=config.EFFICIENCY,
                fee_per_mwh=config.FEE_PER_MWH
            )
            
            # Threshold-Lookahead-Strategie anwenden mit spezifischer Window Size
            storage, energy_history = threshold_lookahead(
                prices_df, 
                storage, 
                window_size=window_size
            )
            
            # Statistiken berechnen
            total_transactions = len(storage.transactions)
            total_charged = sum([t.get('amount', 0) for t in storage.transactions if t['type'] == 'charge'])
            total_discharged = sum([t.get('amount_gross', t.get('amount', 0)) for t in storage.transactions if t['type'] == 'discharge'])
            total_profit = storage.cash
            total_cycles = storage.total_cycles
            
            # Berechnung der Handelspreisspreizung
            if total_transactions > 0:
                charge_prices = [t['price'] for t in storage.transactions if t['type'] == 'charge']
                discharge_prices = [t['price'] for t in storage.transactions if t['type'] == 'discharge']
                
                avg_charge_price = np.mean(charge_prices) if charge_prices else 0
                avg_discharge_price = np.mean(discharge_prices) if discharge_prices else 0
                price_spread = avg_discharge_price - avg_charge_price
            else:
                avg_charge_price = 0
                avg_discharge_price = 0
                price_spread = 0
            
            # Ergebnis für diese Window Size speichern
            result = {
                'window_size': window_size,
                'window_minutes': window_size * 15,
                'window_hours': window_size / 4,  # 15 min = 1/4 Stunde
                'profit': total_profit,
                'transactions': total_transactions,
                'charge_transactions': len([t for t in storage.transactions if t['type'] == 'charge']),
                'discharge_transactions': len([t for t in storage.transactions if t['type'] == 'discharge']),
                'total_charged': total_charged,
                'total_discharged': total_discharged,
                'cycles': total_cycles,
                'avg_charge_price': avg_charge_price,
                'avg_discharge_price': avg_discharge_price,
                'price_spread': price_spread,
                'final_energy_level': storage.energy_level
            }
            
            results.append(result)
            
            # Ergebnisse im Log anzeigen
            logger.info(f"Ergebnis für Window Size {window_size}:")
            logger.info(f"  - Profit: {total_profit:,.2f} €")
            logger.info(f"  - Anzahl Transaktionen: {total_transactions}")
            logger.info(f"  - Zyklen: {total_cycles:.2f}")
            logger.info(f"  - Ø Kaufpreis: {avg_charge_price:.2f} €/MWh")
            logger.info(f"  - Ø Verkaufspreis: {avg_discharge_price:.2f} €/MWh")
            logger.info(f"  - Preisspreizung: {price_spread:.2f} €/MWh")
            
            # Konfiguration für diese Simulation speichern
            save_config_snapshot(window_dir, window_size)
            
            # Optional: Detaillierte Analyse für diese Window Size durchführen
            # (Hier könntest du zusätzliche Analysen für jede Window Size durchführen)
            
            # Ausführungszeit für diese Simulation
            elapsed_time = time.time() - start_time
            logger.info(f"Simulation für Window Size {window_size} abgeschlossen in {elapsed_time:.2f} Sekunden")
            
        except Exception as e:
            logger.error(f"Fehler bei Window Size {window_size}: {str(e)}")
            # Füge trotzdem einen Eintrag mit Fehlerstatus hinzu
            results.append({
                'window_size': window_size,
                'window_minutes': window_size * 15,
                'window_hours': window_size / 4,
                'profit': 0,
                'transactions': 0,
                'cycles': 0,
                'error': str(e)
            })
    
    # Ergebnisse in DataFrame für einfachere Analyse konvertieren
    results_df = pd.DataFrame(results)
    
    # Ergebnisse speichern
    results_csv_path = os.path.join(base_output_dir, "window_size_results.csv")
    results_df.to_csv(results_csv_path, index=False)
    
    # Visualisierung der Ergebnisse
    create_results_visualization(results_df, base_output_dir)
    
    # Gesamtlaufzeit berechnen
    total_elapsed_time = time.time() - total_start_time
    hours, remainder = divmod(total_elapsed_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    logger.info(f"\nGesamtlaufzeit: {int(hours):02d}:{int(minutes):02d}:{seconds:.2f}")
    
    logger.info(f"Analyse abgeschlossen. Ergebnisse gespeichert in: {base_output_dir}")
    
    return results_df, base_output_dir

def save_config_snapshot(output_dir, window_size):
    """Speichert einen Snapshot der Konfiguration mit der aktuellen Window Size."""
    import inspect
    
    config_file_path = os.path.join(output_dir, "config_snapshot.py")
    
    with open(config_file_path, 'w', encoding='utf-8') as f:
        f.write("# Konfiguration zum Zeitpunkt der Simulation\n")
        f.write(f"# Window Size: {window_size}\n\n")
        
        # Alle Attribute aus config auslesen, die keine Funktionen sind
        for name, value in inspect.getmembers(config):
            if not name.startswith('__') and not inspect.isfunction(value) and not inspect.ismodule(value):
                if name == 'ROLLING_WINDOW_SIZE':
                    # Überschreibe die Standardfenstergröße mit der aktuellen
                    f.write(f"{name} = {window_size}  # Überschrieben für diese Simulation\n")
                elif isinstance(value, str):
                    f.write(f"{name} = '{value}'\n")
                else:
                    f.write(f"{name} = {value}\n")

def create_results_visualization(results_df, output_dir):
    """Erstellt Visualisierungen der Ergebnisse."""
    logger = logging.getLogger(__name__)
    
    if results_df.empty:
        logger.warning("Keine Ergebnisse für Visualisierung vorhanden.")
        return
    
    try:
        # 1. Profit vs. Window Size - Hauptvisualisierung für die Forschungsfrage
        plt.figure(figsize=(12, 8))
        
        # Stunden als X-Achse für bessere Lesbarkeit
        plt.subplot(2, 1, 1)
        plt.plot(results_df['window_hours'], results_df['profit'], 'o-', 
                 linewidth=2, markersize=8, color='#1F77B4')
        
        # Markiere den optimalen Punkt
        if not results_df['profit'].isna().all():
            optimal_idx = results_df['profit'].idxmax()
            optimal_window = results_df.loc[optimal_idx, 'window_hours']
            optimal_profit = results_df.loc[optimal_idx, 'profit']
            
            plt.axvline(x=optimal_window, color='red', linestyle='--', 
                      label=f'Optimum: {optimal_window:.2f} h')
            plt.scatter([optimal_window], [optimal_profit], color='red', s=100, zorder=5)
            
            # Beschriftung für optimalen Punkt
            plt.annotate(f'Optimaler Horizont: {optimal_window:.2f} h\nGewinn: {optimal_profit:,.2f} €',
                       xy=(optimal_window, optimal_profit),
                       xytext=(optimal_window + 0.5, optimal_profit * 1.05),
                       arrowprops=dict(arrowstyle='->'))
        
        plt.grid(True, alpha=0.3)
        plt.title('Einfluss des Rolling-Window-Horizonts auf den Arbitragegewinn', 
                 fontsize=14, fontweight='bold')
        plt.xlabel('Horizont (Stunden)', fontsize=12)
        plt.ylabel('Arbitragegewinn (€)', fontsize=12)
        plt.legend()
        
        # 2. Zusätzliche Metriken: Transaktionen und Zyklen
        plt.subplot(2, 1, 2)
        plt.plot(results_df['window_hours'], results_df['transactions'], 'o-', 
                label='Anzahl Trades', color='#FF7F0E')
        plt.plot(results_df['window_hours'], results_df['cycles'], 'o-', 
                label='Zyklen', color='#2CA02C')
        plt.grid(True, alpha=0.3)
        plt.xlabel('Horizont (Stunden)', fontsize=12)
        plt.ylabel('Anzahl', fontsize=12)
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'window_size_analysis.png'), 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # 3. Preisspreizung vs. Window Size
        plt.figure(figsize=(10, 6))
        plt.plot(results_df['window_hours'], results_df['price_spread'], 'o-', 
                linewidth=2, color='#9467BD')
        plt.grid(True, alpha=0.3)
        plt.title('Einfluss des Horizonts auf die erzielte Preisspreizung', 
                 fontsize=14, fontweight='bold')
        plt.xlabel('Horizont (Stunden)', fontsize=12)
        plt.ylabel('Durchschnittliche Preisspreizung (€/MWh)', fontsize=12)
        plt.savefig(os.path.join(output_dir, 'price_spread_analysis.png'), 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # 4. Gewinn pro Zyklus vs. Window Size
        plt.figure(figsize=(10, 6))
        
        # Berechne Gewinn pro Zyklus (mit Überprüfung auf Null-Division)
        results_df['profit_per_cycle'] = np.where(
            results_df['cycles'] > 0,
            results_df['profit'] / results_df['cycles'],
            0
        )
        
        plt.plot(results_df['window_hours'], results_df['profit_per_cycle'], 'o-', 
                linewidth=2, color='#D62728')
        plt.grid(True, alpha=0.3)
        plt.title('Effizienz: Gewinn pro Batteriezyklus nach Horizont', 
                 fontsize=14, fontweight='bold')
        plt.xlabel('Horizont (Stunden)', fontsize=12)
        plt.ylabel('Gewinn pro Zyklus (€/Zyklus)', fontsize=12)
        plt.savefig(os.path.join(output_dir, 'profit_per_cycle_analysis.png'), 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info("Visualisierungen der Ergebnisse erstellt.")
        
    except Exception as e:
        logger.error(f"Fehler bei der Visualisierung der Ergebnisse: {str(e)}")

# Wenn direkt ausgeführt (nicht importiert)
if __name__ == "__main__":
    results_df, output_dir = test_window_sizes()
    
    # Zeige die optimale Window Size an
    if not results_df.empty and not results_df['profit'].isna().all():
        optimal_idx = results_df['profit'].idxmax()
        optimal_window = results_df.loc[optimal_idx, 'window_size']
        optimal_profit = results_df.loc[optimal_idx, 'profit']
        
        print(f"\n{'='*80}")
        print(f"OPTIMALER ROLLING-WINDOW-HORIZONT: {optimal_window} (= {optimal_window*15} Minuten)")
        print(f"MAXIMALER ARBITRAGEGEWINN: {optimal_profit:,.2f} €")
        print(f"{'='*80}")

2025-07-31 11:35:01,690 - INFO - Starte Analyse verschiedener Window Sizes für die Threshold-Lookahead-Strategie
2025-07-31 11:35:01,691 - INFO - Starte Einlesen der Datei: C:\Users\tillw\OneDrive\Bachelorarbeit\00_Großhandelspreise\SMARD_15min_Jahr 2024_DE\Gro_handelspreise_202401010000_202501010000_Viertelstunde.csv
2025-07-31 11:35:01,725 - INFO - Erfolgreich eingelesen: 35136 Zeilen
2025-07-31 11:35:01,867 - INFO - Zeitreihe konvertiert: 2024-01-01 00:00:00 bis 2024-12-31 23:45:00
2025-07-31 11:35:01,871 - INFO - 
Aufbereitete Daten:
2025-07-31 11:35:01,873 - INFO - Zeitraum: 01.01.2024 bis 31.12.2024
2025-07-31 11:35:01,875 - INFO - Anzahl der Datenpunkte: 35136
2025-07-31 11:35:01,876 - INFO - Preisbereich: -135.45 € bis 936.28 €/MWh
2025-07-31 11:35:01,878 - INFO - Durchschnittspreis: 78.51 €/MWh
2025-07-31 11:35:01,879 - INFO - Preisvolatilität (Stdabw.): 52.72 €/MWh
2025-07-31 11:35:01,890 - INFO - Anzahl eindeutiger Tage im Datensatz: 366
2025-07-31 11:35:01,891 - INFO - 
202


OPTIMALER ROLLING-WINDOW-HORIZONT: 6 (= 90 Minuten)
MAXIMALER ARBITRAGEGEWINN: 19,733.69 €
